In [1]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import *

In [2]:
import tensorflow as tf

In [3]:
from _context import dl_uncertainty
import dl_uncertainty.tf_utils as tf_utils
import dl_uncertainty.tf_utils.layers as layers

In [4]:
@layers.scoped
def dropout(x):
    #v= tf.get_variable("var", initializer=tf.random_uniform([]))
    return tf.nn.dropout(x, keep_prob=0.5)

In [5]:
x = np.array([[1,2],[3,4.0]])

In [38]:
ys = layers.sample(lambda x: dropout(x, name='do'), x, 30)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x
ys2 = layers.sample(lambda x: dropout(x, name='do'), x, 30, examplewise=True)

In [39]:
sess = tf.Session()

In [40]:
sess.run(tf.global_variables_initializer())

In [49]:
print(x.shape)
print('Sampling')
r = sess.run(ys)
print(r[:4], '...')
print(r[-1:])
print('shape:', r.shape)

print('')
print('Sample mean')
r = sess.run(y_mean)
print(r)

print('')
print('Example-wise sampling')
r = sess.run(ys2)
print(r[:4], '...')
print(r[-1:])
print('shape:', r.shape)

(2, 2)
Sampling
[[[0. 4.]
  [6. 8.]]

 [[0. 4.]
  [6. 8.]]

 [[2. 0.]
  [0. 8.]]

 [[2. 0.]
  [0. 8.]]] ...
[[[2. 4.]
  [0. 0.]]]
shape: (6, 2, 2)

Sample mean
[[1.33333333 2.66666667]
 [4.         5.33333333]]

Example-wise sampling
[[[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 4.]
  [0. 0.]
  [2. 4.]]

 [[0. 0.]
  [0. 8.]
  [6. 8.]
  [0. 8.]
  [0. 0.]
  [0. 0.]]] ...
[[[0. 0.]
  [0. 8.]
  [6. 8.]
  [0. 8.]
  [0. 0.]
  [0. 0.]]]
shape: (2, 6, 2)
